In [6]:
import os
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
print(tf.test.is_gpu_available())
work_dir = os.getcwd() 
train_dir = os.path.join(work_dir,'train2')
test_dir = os.path.join(work_dir,'test2')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True) 

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True


In [3]:
train_ratio=0.8
test_ratio=0.2
dir_list=os.listdir(os.path.join(work_dir ,'train_cat/train_cat'))

In [4]:
from shutil import copyfile
import numpy as np
source_dir = os.path.join(work_dir ,'train_cat/train_cat')

dir_list = os.listdir(source_dir)

for folder in dir_list:
    data_dir = os.listdir(os.path.join(source_dir,folder))
    np.random.shuffle(data_dir)     
    os.makedirs(os.path.join(train_dir , folder), exist_ok=True)
    os.makedirs(os.path.join(test_dir , folder), exist_ok=True)
    train_data = data_dir[:int(len(data_dir)*train_ratio+1)]
    test_data = data_dir[-int(len(data_dir)*test_ratio):]
  
    for image in train_data:
        copyfile(os.path.join(source_dir,folder,image) , os.path.join(train_dir,folder,image)) 
    
    for image in test_data:
       copyfile(os.path.join(source_dir,folder,image) , os.path.join(test_dir,folder,image))
    

In [64]:
trained_model=MobileNetV2(input_shape=(160,160,3),include_top=False,weights='imagenet')   
trained_model.trainable=True
trained_model.summary()

Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 160, 160, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 80, 80, 32)   864         ['input_4[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 80, 80, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 80, 80, 32)   0           ['bn_Conv1[0][

In [65]:
last_layer=trained_model.get_layer('out_relu')   
last_layer_output=last_layer.output

In [67]:
res = tf.keras.layers.GlobalAveragePooling2D()(last_layer_output)
res = tf.keras.layers.Dropout(0.8)(res)                      
res = tf.keras.layers.Dense(100,activation='softmax')(res)

model = tf.keras.Model(trained_model.input,res) 

model.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(1e-4),metrics=['accuracy'])
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 160, 160, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 80, 80, 32)   864         ['input_4[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 80, 80, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 80, 80, 32)   0           ['bn_Conv1[0][0]']         

In [68]:
train_datagen = ImageDataGenerator(rescale=1/255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,rotation_range=40,
                                 width_shift_range=0.1,height_shift_range=0.1)
                                 
test_datagen = ImageDataGenerator(rescale=1/255)

train_generator=train_datagen.flow_from_directory(train_dir,target_size=(160,160),class_mode='categorical')

Found 10887 images belonging to 100 classes.


In [69]:
test_generator=test_datagen.flow_from_directory(test_dir,target_size=(160,160),class_mode='categorical')

Found 2664 images belonging to 100 classes.


In [70]:
result=model.fit(train_generator,validation_data=test_generator,epochs=20,verbose=1)

Epoch 1/20
341/341 [==============================] - 218s 627ms/step - loss: 5.0730 - accuracy: 0.0148 - val_loss: 4.9629 - val_accuracy: 0.0191
Epoch 2/20
341/341 [==============================] - 135s 396ms/step - loss: 4.5930 - accuracy: 0.0226 - val_loss: 4.5736 - val_accuracy: 0.0319
Epoch 3/20
341/341 [==============================] - 136s 399ms/step - loss: 4.4466 - accuracy: 0.0340 - val_loss: 4.4826 - val_accuracy: 0.0390
Epoch 4/20
341/341 [==============================] - 135s 396ms/step - loss: 4.2657 - accuracy: 0.0531 - val_loss: 4.0162 - val_accuracy: 0.0878
Epoch 5/20
341/341 [==============================] - 134s 393ms/step - loss: 4.0099 - accuracy: 0.0854 - val_loss: 3.7140 - val_accuracy: 0.1449
Epoch 6/20
341/341 [==============================] - 132s 388ms/step - loss: 3.7421 - accuracy: 0.1281 - val_loss: 3.4013 - val_accuracy: 0.2057
Epoch 7/20
341/341 [==============================] - 128s 375ms/step - loss: 3.4785 - accuracy: 0.1779 - val_loss: 3.1236 -

In [72]:
images = [(r"test_crop2/" + f) for f in os.listdir(r"test_crop2/")]
predictions = []
import pandas as pd
for path in images:
    classes=train_generator.class_indices
    class_names=list(classes.keys())
    img = tf.keras.preprocessing.image.load_img(
        path, target_size=(160, 160)
    )

    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    img_array=img_array/255.

    score = model.predict(img_array)
    print(os.path.splitext(path[11:])[0])
    predicted_class_name = class_names[np.argmax(score)][:].title()
    predictions.append((os.path.splitext(path[11:])[0],predicted_class_name))

predictions_df = pd.DataFrame(predictions, columns=['Id','Category'])

predictions_df.to_csv('predictions.csv', index=False)

1/1 [==============================] - 1s 832ms/step
0
The Person in the Image is Anna Friel .
1/1 [==============================] - 0s 31ms/step
1
The Person in the Image is Alice Eve .
1/1 [==============================] - 0s 29ms/step
10
The Person in the Image is Anna Faris .
1/1 [==============================] - 0s 30ms/step
100
The Person in the Image is Adriana Lima .
1/1 [==============================] - 0s 33ms/step
1000
The Person in the Image is Anne Hathaway .
1/1 [==============================] - 0s 32ms/step
1001
The Person in the Image is Benjamin Bratt .
1/1 [==============================] - 0s 32ms/step
1002
The Person in the Image is Annette Bening .
1/1 [==============================] - 0s 31ms/step
1003
The Person in the Image is Benjamin Bratt .
1/1 [==============================] - 0s 31ms/step
1004
The Person in the Image is Barbra Streisand .
1/1 [==============================] - 0s 31ms/step
1005
The Person in the Image is Anthony Perkins .
1/1 [======